In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import time, enum, math
import numpy as np
import pandas as pd
import pylab as plt
from networkx.algorithms.shortest_paths.generic import has_path
import networkx as nx
import random
from tqdm import tqdm, trange

data_path = '/content/drive/MyDrive/_GEO-PYTHON/' #set to wherever the data files are, will be used on every input

## Data Imports

In [23]:
route_blockages = pd.read_csv(('/content/drive/MyDrive/Sim & Modeling/Data/route_blockage.csv'))

distances_df = pd.read_csv((data_path + 'distances.csv'))
distances = distances_df[["prev_port", "next_port", "distance"]]
distances.astype({'prev_port':'int64', 'next_port':'int64'}).dtypes


prev_port      int64
next_port      int64
distance     float64
dtype: object

In [24]:
route_blockages

,Unnamed: 0,prev_port,next_port,affected_by_gibraltar,affected_by_malacca,affected_by_dover,affected_by_suez,affected_by_hormuz,affected_by_panama
0,0,26,1940,False,False,False,False,False,False
1,1,26,1973,False,False,False,False,False,False
2,2,26,1975,False,False,False,False,False,False
3,3,26,2005,False,False,False,False,False,False
4,4,26,2403,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
40199,40199,49142,23968,False,False,False,False,False,False
40200,40200,49142,23979,False,False,False,False,False,False
40201,40201,49142,49085,False,False,False,False,False,False
40202,40202,49159,3281,False,False,False,False,False,False


Since there 

In [25]:
clean_distances = distances.sort_values(by=['prev_port', 'next_port', 'distance'])
clean_distances.drop_duplicates(subset=['prev_port', 'next_port'], keep='first', inplace=True, ignore_index=True)



In [26]:
# clean_distances.to_csv((data_path + 'clean_distances.csv'))

In [27]:
G = nx.from_pandas_edgelist(clean_distances, "prev_port", "next_port", ["distance"], create_using=nx.Graph())


## Create Pruning Plans

When cutting the network, we want to ensure that we cut the correct number of edges 
(some port combinations have more than one edge connecting them, we assume that if one is blocked, all are blocked)

We therefore create a list indicating the number of edges, and then separate the file by 


In [28]:

route_blockages_gib =  route_blockages[route_blockages['affected_by_gibraltar']==True]
route_blockages_mal =  route_blockages[route_blockages['affected_by_malacca']==True]
route_blockages_dov =  route_blockages[route_blockages['affected_by_dover']==True]
route_blockages_suez =  route_blockages[route_blockages['affected_by_suez']==True]
route_blockages_horm =  route_blockages[route_blockages['affected_by_hormuz']==True]
route_blockages_pan = route_blockages[route_blockages['affected_by_panama']==True]

route_blockages_horm = route_blockages_horm[["prev_port", "next_port"]]
route_blockages_gib =  route_blockages_gib[["prev_port", "next_port"]]
route_blockages_mal =  route_blockages_mal[["prev_port", "next_port"]]
route_blockages_dov =  route_blockages_dov[["prev_port", "next_port"]]
route_blockages_suez =  route_blockages_suez[["prev_port", "next_port"]]
route_blockages_pan = route_blockages_pan[["prev_port", "next_port"]]



In [29]:
route_blockages_total = pd.concat([route_blockages_horm, route_blockages_gib, route_blockages_mal, route_blockages_dov, route_blockages_suez, route_blockages_pan])

In [30]:
"""
Commented out for your convenience, files are included
"""
# route_blockages_horm.to_csv((data_path + 'route_blockages_horm.csv'))
# route_blockages_mal.to_csv((data_path + 'route_blockages_mal.csv'))
# route_blockages_dov.to_csv((data_path + 'route_blockages_dov.csv'))
# route_blockages_suez.to_csv((data_path + 'route_blockages_suez.csv'))
# route_blockages_gib.to_csv((data_path + 'route_blockages_gib.csv'))
# route_blockages_pan.to_csv((data_path + 'route_blockages_pan.csv'))
# route_blockages_total.to_csv((data_path + 'route_blockages_total.csv'))

'\nCommented out for your convenience, files are included\n'

In [31]:
# route_blockages_horm = pd.read_csv((data_path + 'route_blockages_horm.csv'))

In [32]:
def Cut_Graph(G, route_blockages):
    print('Before Pruning',G.number_of_edges())
    for index in tqdm(range(len(route_blockages)), desc='Removing Edges', delay=0.5):
        try:
            G.remove_edge(route_blockages.iloc[index]['prev_port'],route_blockages.iloc[index]['next_port'])
        except:
            pass
    print('\n', 'After Pruning',G.number_of_edges())
    return G

In [33]:
def Build_Cut_Graph(distances, route_blockages):
    G = nx.from_pandas_edgelist(distances, "prev_port", "next_port", ["distance"], create_using=nx.Graph())
    print('Before Pruning:','\n','Nodes: ',G.number_of_nodes(), 'Edges: ',G.number_of_edges())
    for index in tqdm(range(len(route_blockages)), desc='Removing Edges'):
        try:
            G.remove_edge(route_blockages.iloc[index]['prev_port'],route_blockages.iloc[index]['next_port'])
        except:
            pass
    print('\n', 'After Pruning:','\n', 'Nodes: ',G.number_of_nodes(), 'Edges: ',G.number_of_edges())
    return G

## Preprocessing

In [34]:
import csv
from operator import itemgetter
import networkx as nx
from networkx.algorithms import community
import pandas as pd


#distances_df = pd.read_csv('/content/drive/MyDrive/_GEO-PYTHON/distances.csv')
ports_df = pd.read_csv('/content/drive/MyDrive/_GEO-PYTHON/ports.csv')

In [35]:
ports_df["coords"] = ports_df["coords"].apply(lambda x: eval(x)[0])
ports_df["x"] = ports_df["coords"].apply(lambda x: x[0])
ports_df["y"] = ports_df["coords"].apply(lambda x: x[1])

In [36]:
df1 = clean_distances.merge(ports_df, left_on='prev_port', right_on='Unnamed: 0')
df1 = df1.drop(columns = ["Unnamed: 0","INDEX_NO"])
df1 = df1.rename(columns={"PORT_NAME":"prev_port_name"})
df = df1.merge(ports_df, left_on='next_port', right_on='Unnamed: 0')
df = df.drop(columns = ["Unnamed: 0", "INDEX_NO", "coords_x", "coords_y"])
df = df.rename(columns={"PORT_NAME":"next_port_name", "x_x":"x_prev", "y_x":"y_prev", "x_y":"x_next", "y_y":"y_next"})

In [37]:
df["prev_port_name"] = df["prev_port"].astype(str) + '_' + df["prev_port_name"].astype(str)
df["next_port_name"] = df["next_port"].astype(str) + '_' + df["next_port_name"].astype(str)
df

,prev_port,next_port,distance,prev_port_name,x_prev,y_prev,next_port_name,x_next,y_next
0,26,1940,10030.106970,26_Victoria,-123.327152,48.402783,1940_QINGDAO GANG,120.221339,36.111458
1,537,1940,8834.085377,537_Victoria,-123.449371,48.403568,1940_QINGDAO GANG,120.221339,36.111458
2,756,1940,4975.704274,756_Phatthaya Harbour,100.770954,12.916741,1940_QINGDAO GANG,120.221339,36.111458
3,1550,1940,397.462806,1550_Tangshan Gang,119.067129,38.991341,1940_QINGDAO GANG,120.221339,36.111458
4,1812,1940,123.369544,1812_Lianyungang Gang,119.924884,34.872131,1940_QINGDAO GANG,120.221339,36.111458
...,...,...,...,...,...,...,...,...,...
39929,29534,2337,4174.924280,29534_nan,-48.486450,-1.095905,2337_STRAUMSVIK,-22.054061,64.060591
39930,29572,3975,6.982595,29572_No1,111.770408,21.452923,3975_yangxi power station,111.683047,21.536085
39931,43317,43319,1.937409,43317_Container Yard,120.312885,22.558125,43319_Jhong Jhou Fishing Harbour,120.300516,22.574527
39932,43447,43448,1.159382,43447_Offshore Container Terminal,72.853660,18.943173,43448_nan,72.848746,18.956699


In [38]:
# create port node to update the nodes with the coordinates

prev_port = df[['prev_port', 'prev_port_name', 'x_prev',	'y_prev']].rename(columns = {'prev_port':'port_id', 'prev_port_name':'port_name', 'x_prev':'x', 'y_prev':'y'})
next_port = df[['next_port', 'next_port_name', 'x_next',	'y_next']].rename(columns = {'next_port':'port_id', 'next_port_name':'port_name', 'x_next':'x', 'y_next':'y'})
port_node = prev_port
port_node = port_node.append(next_port, ignore_index=True)
port_node = port_node.drop_duplicates(subset=['port_name'], ignore_index=True)
port_node

,port_id,port_name,x,y
0,26,26_Victoria,-123.327152,48.402783
1,537,537_Victoria,-123.449371,48.403568
2,756,756_Phatthaya Harbour,100.770954,12.916741
3,1550,1550_Tangshan Gang,119.067129,38.991341
4,1812,1812_Lianyungang Gang,119.924884,34.872131
...,...,...,...,...
2907,14852,14852_nan,109.193798,19.727596
2908,2535,2535_riverhead,-72.627874,40.993215
2909,3162,3162_nynashamn,17.969714,58.924710
2910,2337,2337_STRAUMSVIK,-22.054061,64.060591


In [39]:
def get_top_keys(dictionary, top):
    items = dictionary.items()
    items = sorted(items, reverse=True, key=lambda x: x[1])
    return map(lambda x: x[0], items[:top])

In [40]:
# to update the nodes with coordinate if needed
'''
for i, node in port_node.iterrows():
    G.nodes[node['port_name']].update(node[1:].to_dict())
G.nodes(data=True)
'''

"\nfor i, node in port_node.iterrows():\n    G.nodes[node['port_name']].update(node[1:].to_dict())\nG.nodes(data=True)\n"

## No Blockage


In [ ]:
G = nx.from_pandas_edgelist(df, "prev_port_name", "next_port_name", ["distance"], create_using=nx.Graph())

In [ ]:
G.graph['name'] = 'No Blockages'

In [ ]:
info = nx.info(G)
print(info)

Name: No Blockages
Type: Graph
Number of nodes: 2912
Number of edges: 29838
Average degree:  20.4931


In [ ]:
df_nodes = pd.DataFrame(data = dict(G.degree()), index = range(1)).T
df_nodes = df_nodes.reset_index()
df_nodes.columns = ['nodes', 'k']
df_nodes = df_nodes.sort_values('k', ascending=False)
df_nodes[:10]

,nodes,k
11,1975_TANJUNG PELEPAS,884
19,2005_Shanghai Gang,479
450,2787_EUROPA POINT,437
56,3128_HONG KONG,400
76,6868_nan,371
10,1973_JOHOR,357
50,2667_fujairah,341
433,2560_EUROPOORT,338
455,2866_HANSWEERT,278
82,7007_GWANGYANG HANG,271


#### SHORTEST PATHS

In [ ]:
print(nx.dijkstra_path_length(G, '2998_naples' ,'2050_Port of Mumbai', weight = 'distance'))
print(nx.dijkstra_path(G, '2998_naples' ,'2050_Port of Mumbai', weight = 'distance'))

694.9865229473677
['2998_naples', '4834_villa san giovanni', '4154_MARSAXLOKK', '7026_nan', '1594_Tekirdag Port', '3026_NEMRUT LIMANI BAY', '1592_Izmir Liman', '2111_Richards Bay', '2052_DURBAN', '6869_nan', '6859_nan', '1975_TANJUNG PELEPAS', '2109_Port Dahej', '29843_nan', '2050_Port of Mumbai']


In [ ]:
conn_groups = list(nx.connected_components(G))
len(conn_groups)

37

In [ ]:
subcomponent = max(conn_groups, key=len)
sub_G = nx.subgraph(G, subcomponent)
len(subcomponent)

2855

In [ ]:
avg_shortest_path = nx.average_shortest_path_length(sub_G, weight='distance')
avg_shortest_path

1718.3336822223505

#### CENTRALITY

In [ ]:
bw_cent = nx.betweenness_centrality(sub_G, weight = 'distance')
d_cent = nx.degree_centrality(sub_G)
cl_cent = nx.closeness_centrality(sub_G, distance='distance')
eig_cent = nx.eigenvector_centrality(sub_G, weight='distance')

In [ ]:
top_bw_cent = get_top_keys(bw_cent,10)
top_cl_cent = get_top_keys(cl_cent,10)
top_eig_cent = get_top_keys(eig_cent,10)
top_d_cent = get_top_keys(d_cent, 10)

In [ ]:
print('bw_cent')
print(pd.DataFrame(bw_cent.items()).describe())
print('\nd_cent')
print(pd.DataFrame(d_cent.items()).describe())
print('\ncl_cent')
print(pd.DataFrame(cl_cent.items()).describe())
print('\neig_cent')
print(pd.DataFrame(eig_cent.items()).describe())

bw_cent
                 1
count  2855.000000
mean      0.004668
std       0.024038
min       0.000000
25%       0.000000
50%       0.000007
75%       0.001391
max       0.602350

d_cent
                 1
count  2855.000000
mean      0.007314
std       0.013434
min       0.000350
25%       0.001402
50%       0.003153
75%       0.008059
max       0.309741

cl_cent
                 1
count  2855.000000
mean      0.000731
std       0.000267
min       0.000046
25%       0.000514
50%       0.000776
75%       0.000969
max       0.001099

eig_cent
                  1
count  2.855000e+03
mean   7.058307e-03
std    1.733633e-02
min    5.376758e-26
25%    2.504226e-05
50%    9.558514e-04
75%    6.349871e-03
max    3.718409e-01


In [ ]:
print('Top 10 places for betweenness centrality:')
for node_id in top_bw_cent:
    print(node_id,'         ', bw_cent[node_id])

Top 10 places for betweenness centrality:
6869_nan           0.6023500017562257
6868_nan           0.45630400240123936
1594_Tekirdag Port           0.30252938239073146
2052_DURBAN           0.2812110145555485
2111_Richards Bay           0.2790251891872508
1592_Izmir Liman           0.2784845664615936
3026_NEMRUT LIMANI BAY           0.27630979426124436
1342_Gökçeada Harbour           0.2511134838578307
842_Çanakkale Harbour           0.22416536914756252
2866_HANSWEERT           0.22175332227525288


In [ ]:
print('Top 10 places for degree centrality:')
for node_id in top_d_cent:
    print(node_id,'         ', d_cent[node_id])

Top 10 places for degree centrality:
1975_TANJUNG PELEPAS           0.30974071478626486
2005_Shanghai Gang           0.16783461807988787
2787_EUROPA POINT           0.15311843027330063
3128_HONG KONG           0.1401541695865452
6868_nan           0.12999299229152067
1973_JOHOR           0.1250875963559916
2667_fujairah           0.11948142957252977
2560_EUROPOORT           0.11843027330063069
2866_HANSWEERT           0.09740714786264891
7007_GWANGYANG HANG           0.09495444989488437


In [ ]:
print('Top 10 places for closeness centrality:')
for node_id in top_cl_cent:
    print(node_id,'         ', cl_cent[node_id])

Top 10 places for closeness centrality:
6869_nan           0.0010992358007631557
6868_nan           0.0010991547348735383
2052_DURBAN           0.0010988357185259512
2075_PORT KLANG           0.0010986438634677288
7019_nan           0.0010984782941635205
3419_AL JUBAYL           0.001098382950274083
6859_nan           0.0010983279486707948
7100_nan           0.0010982634574039324
6820_No 1           0.001098142649418982
1975_TANJUNG PELEPAS           0.0010980278721784424


In [ ]:
print('Top 10 places for eigenvector centrality:')
for node_id in top_eig_cent:
    print(node_id,'         ', eig_cent[node_id])

Top 10 places for eigenvector centrality:
1975_TANJUNG PELEPAS           0.37184085528199495
6868_nan           0.23469318384240156
6869_nan           0.1891433247100044
3099_RESERVE           0.17806495713144804
2667_fujairah           0.14871669116312294
2787_EUROPA POINT           0.13941922247729205
2560_EUROPOORT           0.13738808131727567
28317_No.3           0.13597015509660573
7144_nan           0.12667434851867293
2110_Unknown           0.12544556698569065


#### CLUSTERING

In [ ]:
#avg of local clustering of a node, which is triangles that actually exist over all possible triangles in its neighborhood. 
avg_clustering = nx.average_clustering(sub_G, weight='distance')
avg_clustering

0.015258297290193448

In [ ]:
# number of pissible triangles
triadic_closure = nx.transitivity(sub_G)
print("Triadic closure:", triadic_closure)

Triadic closure: 0.20638977586576665


#### ISOLATES


In [ ]:
nx.number_of_isolates(sub_G)

0

#### DISTANCE MEASURE

In [ ]:
print('Diameter:{}'.format(nx.diameter(sub_G)))

Diameter:12


## Suez blockage

In [ ]:
# route_blockages_suez = pd.read_csv((data_path + 'route_blockages_suez.csv'))

In [ ]:
G_without_suez = Build_Cut_Graph(df, route_blockages_suez)

Removing Edges: 100%|██████████| 629/629 [00:00<00:00, 5998.20it/s]

Before Pruning: 
 Nodes:  2912 Edges:  29838

 After Pruning: 
 Nodes:  2912 Edges:  29365


In [ ]:
G_without_suez.graph['name'] = 'Suez Blocked'

In [ ]:
info_suez = nx.info(G_without_suez)
print(info_suez)

Name: Suez Blocked
Type: Graph
Number of nodes: 2912
Number of edges: 29365
Average degree:  20.1683


In [ ]:
df_nodes_suez = pd.DataFrame(data = dict(G_without_suez.degree()), index = range(1)).T.reset_index()
df_nodes_suez.columns = ['nodes', 'k']
df_nodes_suez = df_nodes_suez.sort_values('k', ascending=False)
df_nodes_suez[:10]

,nodes,k
11,1975,880
19,2005,478
450,2787,433
56,3128,398
76,6868,369
10,1973,355
50,2667,339
433,2560,335
455,2866,275
82,7007,269


#### SHORTEST PATHS

In [ ]:
print(nx.dijkstra_path_length(G_without_suez, 2998 ,2050, weight = 'distance'))
print(nx.dijkstra_path(G_without_suez, 1550 ,2050, weight = 'distance'))

694.9865229473677
[1550, 23711, 27273, 3962, 27584, 2117, 3855, 3501, 6868, 1975, 2109, 29843, 2050]


In [ ]:
conn_groups_suez = list(nx.connected_components(G_without_suez))
len(conn_groups_suez)

41

In [ ]:
subcomponent_suez = max(conn_groups_suez, key=len)
sub_G_suez = nx.subgraph(G_without_suez, subcomponent_suez)
len(subcomponent_suez)

2851

In [ ]:
avg_shortest_path_suez = nx.average_shortest_path_length(sub_G_suez, weight='distance')
avg_shortest_path_suez

1738.4007539122022

#### CENTRALITY

In [ ]:
bw_cent_suez = nx.betweenness_centrality(sub_G_suez, weight = 'distance')
d_cent_suez = nx.degree_centrality(sub_G_suez)
cl_cent_suez = nx.closeness_centrality(sub_G_suez, distance = 'distance')
eig_cent_suez = nx.eigenvector_centrality(sub_G_suez, weight = 'distance')

In [ ]:
top_bw_cent_suez = get_top_keys(bw_cent_suez,10)
top_cl_cent_suez = get_top_keys(cl_cent_suez,10)
top_eig_cent_suez = get_top_keys(eig_cent_suez,10)
top_d_cent_suez = get_top_keys(d_cent_suez, 10)

In [ ]:
bw_cent_suez.items()

In [ ]:
print('bw_cent')
print(pd.DataFrame(bw_cent_suez.values()).describe())
print('\nd_cent')
print(pd.DataFrame(d_cent_suez.values()).describe())
print('\ncl_cent')
print(pd.DataFrame(cl_cent_suez.values()).describe())
print('\neig_cent')
print(pd.DataFrame(eig_cent_suez.values()).describe())

bw_cent
                 0
count  2851.000000
mean      0.004747
std       0.024534
min       0.000000
25%       0.000000
50%       0.000007
75%       0.001396
max       0.578624

d_cent
                 0
count  2851.000000
mean      0.007218
std       0.013284
min       0.000351
25%       0.001404
50%       0.003158
75%       0.008070
max       0.308772

cl_cent
                 0
count  2851.000000
mean      0.000720
std       0.000262
min       0.000046
25%       0.000509
50%       0.000749
75%       0.000958
max       0.001084

eig_cent
                  0
count  2.851000e+03
mean   7.022372e-03
std    1.736509e-02
min    5.960994e-26
25%    2.433114e-05
50%    9.452263e-04
75%    6.331543e-03
max    3.743828e-01


In [ ]:
nodes_suez = port_node[port_node['port_id'].isin(list(sub_G_suez.nodes))]

for i, node in nodes_suez.iterrows():
    sub_G_suez.nodes[node['port_id']].update(node[1:].to_dict())
sub_G_suez.nodes(data=True)

NodeDataView({26: {'port_name': '26_Victoria', 'x': -123.32715191091728, 'y': 48.402783083729446}, 1940: {'port_name': '1940_QINGDAO GANG', 'x': 120.22133924391858, 'y': 36.1114580135}, 537: {'port_name': '537_Victoria', 'x': -123.4493705745, 'y': 48.403568258}, 756: {'port_name': '756_Phatthaya Harbour', 'x': 100.77095414178388, 'y': 12.916740649000001}, 1550: {'port_name': '1550_Tangshan Gang', 'x': 119.06712856620985, 'y': 38.991341473999995}, 1812: {'port_name': '1812_Lianyungang Gang', 'x': 119.92488383459992, 'y': 34.8721305135}, 1934: {'port_name': '1934_Vancouver', 'x': -123.2597273115, 'y': 49.1586226685}, 1939: {'port_name': '1939_Dalian Gang', 'x': 121.73994207831859, 'y': 38.96239100955427}, 1945: {'port_name': '1945_TIANJIN XIN GANG', 'x': 118.03203396429149, 'y': 38.9173794765}, 1949: {'port_name': '1949_Port of Brisbane', 'x': 153.26802961287123, 'y': -27.0444596093148}, 1973: {'port_name': '1973_JOHOR', 'x': 104.08482446459018, 'y': 1.4279557760000001}, 1975: {'port_nam

In [ ]:
print('Top 10 places for betweenness centrality:')
for node_id in top_bw_cent_suez:
    print(sub_G_suez.nodes[node_id]['port_name'],'         ', bw_cent_suez[node_id])

Top 10 places for betweenness centrality:
6869_nan           0.5786239554660607
6868_nan           0.4447078383920489
1594_Tekirdag Port           0.3477226235120972
2052_DURBAN           0.3114823914823915
2111_Richards Bay           0.3102384955016534
1592_Izmir Liman           0.30987567198093513
3026_NEMRUT LIMANI BAY           0.3020763210236895
1342_Gökçeada Harbour           0.262920199762305
842_Çanakkale Harbour           0.22632650422124106
2866_HANSWEERT           0.22391383865068076


In [ ]:
print('Top 10 places for degree centrality:')
for node_id in top_d_cent_suez:
    print(sub_G_suez.nodes[node_id]['port_name'],'         ', d_cent_suez[node_id])

Top 10 places for degree centrality:
1975_TANJUNG PELEPAS           0.30877192982456136
2005_Shanghai Gang           0.16771929824561402
2787_EUROPA POINT           0.1519298245614035
3128_HONG KONG           0.13964912280701752
6868_nan           0.1294736842105263
1973_JOHOR           0.12456140350877191
2667_fujairah           0.11894736842105262
2560_EUROPOORT           0.1175438596491228
2866_HANSWEERT           0.09649122807017543
7007_GWANGYANG HANG           0.0943859649122807


In [ ]:
print('Top 10 places for closeness centrality:')
for node_id in top_cl_cent_suez:
    print(sub_G_suez.nodes[node_id]['port_name'],'         ', cl_cent_suez[node_id])

Top 10 places for closeness centrality:
6869_nan           0.0010838397319487012
6868_nan           0.0010837598363904095
2052_DURBAN           0.0010835111132339135
2075_PORT KLANG           0.001083264256573499
7019_nan           0.0010831033948750617
3419_AL JUBAYL           0.0010830095142795285
6859_nan           0.0010829636646683747
7100_nan           0.0010828945073592094
6820_No 1           0.001082775896807856
1975_TANJUNG PELEPAS           0.001082671837465979


In [ ]:
print('Top 10 places for eigenvector centrality:')
for node_id in top_eig_cent_suez:
    print(sub_G_suez.nodes[node_id]['port_name'],'         ', eig_cent_suez[node_id])

Top 10 places for eigenvector centrality:
1975_TANJUNG PELEPAS           0.3743828232101863
6868_nan           0.23859726022151798
6869_nan           0.19103582379596662
3099_RESERVE           0.17915297238876415
2667_fujairah           0.14993442072302318
2787_EUROPA POINT           0.14127815022166243
2560_EUROPOORT           0.13769285198654166
28317_No.3           0.1372041605717662
2110_Unknown           0.12713626518527615
7144_nan           0.1255342774033347


#### CLUSTERING

In [ ]:
#avg of local clustering of a node, which is triangles that actually exist over all possible triangles in its neighborhood. 
avg_clust_suez = nx.average_clustering(sub_G_suez, weight='distance')
avg_clust_suez

0.015005546417649375

In [ ]:
# number of pissible triangles
triadic_closure = nx.transitivity(sub_G_suez)
print("Triadic closure:", triadic_closure)

Triadic closure: 0.20433453818568875


#### ISOLATES


In [ ]:
nx.number_of_isolates(G_without_suez)

4

In [ ]:
isolated_suez = nx.isolates(G_without_suez)
print(port_node[port_node['port_id'].isin(list(isolated_suez))])

      port_id        port_name          x          y
2054     4241  4241_al shuqaiq  42.104214  17.593870
2056     5750        5750_DUBA  35.539637  27.560645
2058    10652         10652_W5  32.378556  30.313049
2868     4871      4871_PATRAI  21.728408  38.253760


#### DISTANCE MEASURE

In [ ]:
print('Diameter:{}'.format(nx.diameter(sub_G_suez)))

Diameter:12


## Hormuz blockage

In [ ]:
# route_blockages_horm = pd.read_csv((data_path + 'route_blockages_horm.csv'))

In [ ]:
G_without_hormuz = Build_Cut_Graph(df, route_blockages_horm)

Removing Edges: 100%|██████████| 1156/1156 [00:00<00:00, 6094.41it/s]

Before Pruning: 
 Nodes:  2912 Edges:  29838

 After Pruning: 
 Nodes:  2912 Edges:  28957


In [ ]:
G_without_hormuz.graph['name'] = 'Hormuz Blocked'

In [ ]:
info_hormuz = nx.info(G_without_hormuz)
print(info_hormuz)

Name: Hormuz Blocked
Type: Graph
Number of nodes: 2912
Number of edges: 28957
Average degree:  19.8880


In [ ]:
df_nodes_hormuz = pd.DataFrame(data = dict(G_without_hormuz.degree()), index = range(1)).T.reset_index()
df_nodes_hormuz.columns = ['nodes', 'k']
df_nodes_hormuz = df_nodes_hormuz.sort_values('k', ascending=False)
df_nodes_hormuz[:10]

,nodes,k
11,1975,827
19,2005,476
450,2787,437
56,3128,398
10,1973,353
433,2560,338
76,6868,337
455,2866,278
399,2110,266
82,7007,265


#### SHORTEST PATHS

In [ ]:
#print(nx.dijkstra_path_length(G_without_hormuz, 2998 ,2050, weight = 'distance'))
#print(nx.dijkstra_path(G_without_hormuz, 1550 ,2050, weight = 'distance'))

In [ ]:
conn_groups_hormuz = list(nx.connected_components(G_without_hormuz))
len(conn_groups_hormuz)

43

In [ ]:
subcomponent_hormuz = max(conn_groups_hormuz, key=len)
sub_G_hormuz = nx.subgraph(G_without_hormuz, subcomponent_hormuz)
len(subcomponent_hormuz)

2849

In [ ]:
avg_shortest_path_hormuz = nx.average_shortest_path_length(sub_G_hormuz, weight='distance')
avg_shortest_path_hormuz

1731.4859763365823

#### CENTRALITY

In [ ]:
bw_cent_hormuz = nx.betweenness_centrality(sub_G_hormuz, weight = 'distance')
d_cent_hormuz = nx.degree_centrality(sub_G_hormuz)
cl_cent_hormuz = nx.closeness_centrality(sub_G_hormuz, distance = 'distance')
eig_cent_hormuz = nx.eigenvector_centrality(sub_G_hormuz, weight = 'distance')

In [ ]:
top_bw_cent_hormuz = get_top_keys(bw_cent_hormuz,10)
top_cl_cent_hormuz = get_top_keys(cl_cent_hormuz,10)
top_eig_cent_hormuz = get_top_keys(eig_cent_hormuz,10)
top_d_cent_hormuz = get_top_keys(d_cent_hormuz, 10)

In [ ]:
print('bw_cent')
print(pd.DataFrame(bw_cent_hormuz.values()).describe())
print('\nd_cent')
print(pd.DataFrame(d_cent_hormuz.values()).describe())
print('\ncl_cent')
print(pd.DataFrame(cl_cent_hormuz.values()).describe())
print('\neig_cent')
print(pd.DataFrame(eig_cent_hormuz.values()).describe())

bw_cent
                 0
count  2849.000000
mean      0.004732
std       0.024195
min       0.000000
25%       0.000000
50%       0.000008
75%       0.001393
max       0.582065

d_cent
                 0
count  2849.000000
mean      0.007127
std       0.012990
min       0.000351
25%       0.001404
50%       0.003160
75%       0.007725
max       0.290379

cl_cent
                 0
count  2849.000000
mean      0.000722
std       0.000262
min       0.000046
25%       0.000511
50%       0.000761
75%       0.000952
max       0.001091

eig_cent
                  0
count  2.849000e+03
mean   6.890735e-03
std    1.742483e-02
min    6.183791e-26
25%    1.175169e-05
50%    8.320028e-04
75%    5.864437e-03
max    3.717515e-01


In [ ]:
nodes_hormuz = port_node[port_node['port_id'].isin(list(sub_G_hormuz.nodes))]

for i, node in nodes_hormuz.iterrows():
    sub_G_hormuz.nodes[node['port_id']].update(node[1:].to_dict())
sub_G_hormuz.nodes(data=True)

NodeDataView({26: {'port_name': '26_Victoria', 'x': -123.32715191091728, 'y': 48.402783083729446}, 1940: {'port_name': '1940_QINGDAO GANG', 'x': 120.22133924391858, 'y': 36.1114580135}, 537: {'port_name': '537_Victoria', 'x': -123.4493705745, 'y': 48.403568258}, 756: {'port_name': '756_Phatthaya Harbour', 'x': 100.77095414178388, 'y': 12.916740649000001}, 1550: {'port_name': '1550_Tangshan Gang', 'x': 119.06712856620985, 'y': 38.991341473999995}, 1812: {'port_name': '1812_Lianyungang Gang', 'x': 119.92488383459992, 'y': 34.8721305135}, 1934: {'port_name': '1934_Vancouver', 'x': -123.2597273115, 'y': 49.1586226685}, 1939: {'port_name': '1939_Dalian Gang', 'x': 121.73994207831859, 'y': 38.96239100955427}, 1945: {'port_name': '1945_TIANJIN XIN GANG', 'x': 118.03203396429149, 'y': 38.9173794765}, 1949: {'port_name': '1949_Port of Brisbane', 'x': 153.26802961287123, 'y': -27.0444596093148}, 1973: {'port_name': '1973_JOHOR', 'x': 104.08482446459018, 'y': 1.4279557760000001}, 1975: {'port_nam

In [ ]:
print('Top 10 places for betweenness centrality:')
for node_id in top_bw_cent_hormuz:
    print(sub_G_hormuz.nodes[node_id]['port_name'],'         ', bw_cent_hormuz[node_id])

Top 10 places for betweenness centrality:
6869_nan           0.5820647498056303
6868_nan           0.4865315056653367
1594_Tekirdag Port           0.30288510870895047
2052_DURBAN           0.2813746877256959
2111_Richards Bay           0.27920332066476444
1592_Izmir Liman           0.27866633712601085
3026_NEMRUT LIMANI BAY           0.2765504690527778
1342_Gökçeada Harbour           0.25161341723793623
842_Çanakkale Harbour           0.22456197732286698
2866_HANSWEERT           0.2221486346755702


In [ ]:
print('Top 10 places for degree centrality:')
for node_id in top_d_cent_hormuz:
    print(sub_G_hormuz.nodes[node_id]['port_name'],'         ', d_cent_hormuz[node_id])

Top 10 places for degree centrality:
1975_TANJUNG PELEPAS           0.29037921348314605
2005_Shanghai Gang           0.16713483146067415
2787_EUROPA POINT           0.15344101123595505
3128_HONG KONG           0.13974719101123595
1973_JOHOR           0.12394662921348314
2560_EUROPOORT           0.11867977528089887
6868_nan           0.11832865168539325
2866_HANSWEERT           0.0976123595505618
2110_Unknown           0.09339887640449437
7007_GWANGYANG HANG           0.09304775280898876


In [ ]:
print('Top 10 places for closeness centrality:')
for node_id in top_cl_cent_hormuz:
    print(sub_G_hormuz.nodes[node_id]['port_name'],'         ', cl_cent_hormuz[node_id])

Top 10 places for closeness centrality:
6869_nan           0.0010909552182779344
6868_nan           0.0010908918548707567
2052_DURBAN           0.0010905619201977872
2075_PORT KLANG           0.0010903721644882434
7019_nan           0.0010902092382536809
6859_nan           0.0010900617337383973
7100_nan           0.001089997589956938
6820_No 1           0.001089894931597526
1975_TANJUNG PELEPAS           0.0010897695824974454
2140_Unknown           0.001089674437131628


In [ ]:
print('Top 10 places for eigenvector centrality:')
for node_id in top_eig_cent_hormuz:
    print(sub_G_hormuz.nodes[node_id]['port_name'],'         ', eig_cent_hormuz[node_id])

Top 10 places for eigenvector centrality:
1975_TANJUNG PELEPAS           0.3717515300083259
6868_nan           0.23510353625066843
6869_nan           0.18852611207760844
3099_RESERVE           0.18180678330144048
2787_EUROPA POINT           0.14352308904516528
2560_EUROPOORT           0.14160407066953004
2667_fujairah           0.1378590261291422
28317_No.3           0.136827939251675
2110_Unknown           0.1285079150185274
7144_nan           0.11859579660340455


#### CLUSTERING

In [ ]:
#avg of local clustering of a node, which is triangles that actually exist over all possible triangles in its neighborhood. 
avg_clust_hormuz = nx.average_clustering(sub_G_hormuz, weight='distance')
avg_clust_hormuz

0.01483454046786651

In [ ]:
# number of pissible triangles
triadic_closure = nx.transitivity(sub_G_hormuz)
print("Triadic closure:", triadic_closure)

Triadic closure: 0.2102426043262308


#### ISOLATES


In [ ]:
nx.number_of_isolates(G_without_hormuz)

5

In [ ]:
isolated_hormuz = nx.isolates(G_without_hormuz)
print(port_node[port_node['port_id'].isin(list(isolated_hormuz))])

      port_id              port_name          x          y
575      6946               6946_nan  56.354580  26.955025
1163     2425  2425_Ajman Port Limit  55.447174  25.433304
2786    26940              26940_nan  60.567084  25.287500
2810    22856               22856_D1  51.758299  25.858961
2811    22861               22861_D4  51.808374  25.858333


#### DISTANCE MEASURE

In [ ]:
print('Diameter:{}'.format(nx.diameter(sub_G_hormuz)))

Diameter:14


## Gibraltar Blockage

In [ ]:
# route_blockages_gib = pd.read_csv((data_path + 'route_blockages_gib.csv')) 

In [ ]:
G_without_gibraltar = Build_Cut_Graph(df, route_blockages_gib)

Removing Edges:  29%|██▉       | 613/2099 [00:00<00:00, 6120.49it/s]

Before Pruning: 
 Nodes:  2912 Edges:  29838


Removing Edges: 100%|██████████| 2099/2099 [00:00<00:00, 5924.76it/s]


 After Pruning: 
 Nodes:  2912 Edges:  28164


In [ ]:
G_without_gibraltar.graph['name'] = 'Gibraltar Blocked'

In [ ]:
info_gibraltar = nx.info(G_without_gibraltar)
print(info_gibraltar)

Name: Gibraltar Blocked
Type: Graph
Number of nodes: 2912
Number of edges: 28164
Average degree:  19.3434


In [ ]:
df_nodes_gibraltar = pd.DataFrame(data = dict(G_without_gibraltar.degree()), index = range(1)).T.reset_index()
df_nodes_gibraltar.columns = ['nodes', 'k']
df_nodes_gibraltar = df_nodes_gibraltar.sort_values('k', ascending=False)
df_nodes_gibraltar[:10]

,nodes,k
11,1975,878
19,2005,479
56,3128,400
76,6868,368
10,1973,357
50,2667,341
433,2560,296
82,7007,271
8,1945,260
77,6869,257


#### SHORTEST PATHS

In [ ]:
#print(nx.dijkstra_path_length(G_without_gibraltar, 2998 ,2050, weight = 'distance'))
#print(nx.dijkstra_path(G_without_gibraltar, 1550 ,2050, weight = 'distance'))

In [ ]:
conn_groups_gibraltar = list(nx.connected_components(G_without_gibraltar))
len(conn_groups_gibraltar)

47

In [ ]:
subcomponent_gibraltar = max(conn_groups_gibraltar, key=len)
sub_G_gibraltar = nx.subgraph(G_without_gibraltar, subcomponent_gibraltar)
len(subcomponent_gibraltar)

2845

In [ ]:
avg_shortest_path_gibraltar = nx.average_shortest_path_length(sub_G_gibraltar, weight='distance')
avg_shortest_path_gibraltar

1969.004949162651

#### CENTRALITY

In [ ]:
bw_cent_gibraltar = nx.betweenness_centrality(sub_G_gibraltar, weight = 'distance')
d_cent_gibraltar = nx.degree_centrality(sub_G_gibraltar)
cl_cent_gibraltar = nx.closeness_centrality(sub_G_gibraltar, distance = 'distance')
eig_cent_gibraltar = nx.eigenvector_centrality(sub_G_gibraltar, weight = 'distance')

In [ ]:
top_bw_cent_gibraltar = get_top_keys(bw_cent_gibraltar,10)
top_cl_cent_gibraltar = get_top_keys(cl_cent_gibraltar,10)
top_eig_cent_gibraltar = get_top_keys(eig_cent_gibraltar,10)
top_d_cent_gibraltar = get_top_keys(d_cent_gibraltar, 10)

In [ ]:
bw_cent_gibraltar.values()

dict_values([1.5336110653501356e-05, 0.06658790831776223, 0.006997718877218603, 0.006452297522525531, 0.0007037295936969574, 0.005621426624378578, 0.005625879043600563, 0.01214372607133864, 0.002702123754497562, 0.01848446575669112, 0.06853312080452247, 0.104130954554157, 0.002075816783938442, 0.0035047959975719476, 0.0017651368648933176, 5.8376163132682586e-05, 0.0027367536817796616, 0.0019976520909302737, 0.018585387259056098, 0.062222805983853546, 0.0061502750853009314, 0.0017134393305936115, 0.0005427004318351932, 0.08126135057705827, 0.017909856320431707, 0.12893760824944234, 0.11945395530661584, 0.003504548640948504, 0.0007032348804500703, 0.0014062224042766971, 0.03038504026718473, 0.0032064839096990014, 0.0015242115136592802, 0.05579623355016615, 0.004208030878022018, 0.0021087152148564366, 0.0, 0.0, 0.0007032348804500703, 0.0007522114918918972, 0.004304252604541567, 0.001432194849738272, 0.030157472173616647, 0.02126129121146864, 0.0, 0.0021087152148564366, 0.00081899778022166

In [ ]:
print('bw_cent')
print(pd.DataFrame(bw_cent_gibraltar.values()).describe())
print('\nd_cent')
print(pd.DataFrame(d_cent_gibraltar.values()).describe())
print('\ncl_cent')
print(pd.DataFrame(cl_cent_gibraltar.values()).describe())
print('\neig_cent')
print(pd.DataFrame(eig_cent_gibraltar.values()).describe())

bw_cent
                 0
count  2845.000000
mean      0.004478
std       0.022643
min       0.000000
25%       0.000000
50%       0.000007
75%       0.001405
max       0.563938

d_cent
                 0
count  2845.000000
mean      0.006952
std       0.012615
min       0.000352
25%       0.001406
50%       0.003165
75%       0.007736
max       0.308720

cl_cent
                 0
count  2845.000000
mean      0.000623
std       0.000228
min       0.000046
25%       0.000419
50%       0.000633
75%       0.000840
max       0.000940

eig_cent
                  0
count  2.845000e+03
mean   6.924426e-03
std    1.742564e-02
min    6.617595e-26
25%    8.851377e-06
50%    5.948752e-04
75%    6.121473e-03
max    3.806076e-01


In [ ]:
nodes_gibraltar = port_node[port_node['port_id'].isin(list(sub_G_gibraltar.nodes))]

for i, node in nodes_gibraltar.iterrows():
    sub_G_gibraltar.nodes[node['port_id']].update(node[1:].to_dict())
sub_G_gibraltar.nodes(data=True)

NodeDataView({26: {'port_name': '26_Victoria', 'x': -123.32715191091728, 'y': 48.402783083729446}, 1940: {'port_name': '1940_QINGDAO GANG', 'x': 120.22133924391858, 'y': 36.1114580135}, 537: {'port_name': '537_Victoria', 'x': -123.4493705745, 'y': 48.403568258}, 756: {'port_name': '756_Phatthaya Harbour', 'x': 100.77095414178388, 'y': 12.916740649000001}, 1550: {'port_name': '1550_Tangshan Gang', 'x': 119.06712856620985, 'y': 38.991341473999995}, 1812: {'port_name': '1812_Lianyungang Gang', 'x': 119.92488383459992, 'y': 34.8721305135}, 1934: {'port_name': '1934_Vancouver', 'x': -123.2597273115, 'y': 49.1586226685}, 1939: {'port_name': '1939_Dalian Gang', 'x': 121.73994207831859, 'y': 38.96239100955427}, 1945: {'port_name': '1945_TIANJIN XIN GANG', 'x': 118.03203396429149, 'y': 38.9173794765}, 1949: {'port_name': '1949_Port of Brisbane', 'x': 153.26802961287123, 'y': -27.0444596093148}, 1973: {'port_name': '1973_JOHOR', 'x': 104.08482446459018, 'y': 1.4279557760000001}, 1975: {'port_nam

In [ ]:
print('Top 10 places for betweenness centrality:')
for node_id in top_bw_cent_gibraltar:
    print(sub_G_gibraltar.nodes[node_id]['port_name'],'         ', bw_cent_gibraltar[node_id])

Top 10 places for betweenness centrality:
6869_nan           0.5639382241674347
6868_nan           0.45321669973824724
1594_Tekirdag Port           0.2934567247113719
7026_nan           0.2761036681503117
2052_DURBAN           0.27159781989766363
2111_Richards Bay           0.2694616480976049
1592_Izmir Liman           0.26894492011123133
3026_NEMRUT LIMANI BAY           0.2671100286785269
7144_nan           0.1935926719116165
6859_nan           0.18323263445192944


In [ ]:
print('Top 10 places for degree centrality:')
for node_id in top_d_cent_gibraltar:
    print(sub_G_gibraltar.nodes[node_id]['port_name'],'         ', d_cent_gibraltar[node_id])

Top 10 places for degree centrality:
1975_TANJUNG PELEPAS           0.30872011251758086
2005_Shanghai Gang           0.16842475386779185
3128_HONG KONG           0.14064697609001406
6868_nan           0.12939521800281295
1973_JOHOR           0.12552742616033755
2667_fujairah           0.11990154711673699
2560_EUROPOORT           0.10407876230661041
7007_GWANGYANG HANG           0.09528832630098454
1945_TIANJIN XIN GANG           0.09142053445850915
6869_nan           0.09036568213783404


In [ ]:
print('Top 10 places for closeness centrality:')
for node_id in top_cl_cent_gibraltar:
    print(sub_G_gibraltar.nodes[node_id]['port_name'],'         ', cl_cent_gibraltar[node_id])

Top 10 places for closeness centrality:
6869_nan           0.000939635507667764
6868_nan           0.0009395787533443014
2052_DURBAN           0.0009393485060766591
2075_PORT KLANG           0.0009392029488538972
7019_nan           0.0009390821438641722
3419_AL JUBAYL           0.0009390147438573483
6859_nan           0.0009390042478860805
7100_nan           0.0009389251262148121
6820_No 1           0.0009388391191745536
1975_TANJUNG PELEPAS           0.0009387958731643619


In [ ]:
print('Top 10 places for eigenvector centrality:')
for node_id in top_eig_cent_gibraltar:
    print(sub_G_gibraltar.nodes[node_id]['port_name'],'         ', eig_cent_gibraltar[node_id])

Top 10 places for eigenvector centrality:
1975_TANJUNG PELEPAS           0.3806076469874121
6868_nan           0.2248191015494407
6869_nan           0.1886766826959961
3099_RESERVE           0.16458692816293252
2667_fujairah           0.16072123685687775
28317_No.3           0.1379185210442175
2560_EUROPOORT           0.132518511911423
1973_JOHOR           0.127986410594394
2110_Unknown           0.12747406100177464
3637_TUBARAO           0.11180371699837402


#### CLUSTERING

In [ ]:
#avg of local clustering of a node, which is triangles that actually exist over all possible triangles in its neighborhood. 
avg_clust_gibraltar = nx.average_clustering(sub_G_gibraltar, weight='distance')
avg_clust_gibraltar

0.014309711039037639

In [ ]:
# number of pissible triangles
triadic_closure = nx.transitivity(sub_G_gibraltar)
print("Triadic closure:", triadic_closure)

Triadic closure: 0.2098997678551853


#### ISOLATES


In [ ]:
nx.number_of_isolates(G_without_gibraltar)

9

In [ ]:
isolated_gibraltar = nx.isolates(G_without_gibraltar)
print(port_node[port_node['port_id'].isin(list(isolated_gibraltar))])

      port_id                   port_name          x          y
450      2787           2787_EUROPA POINT  -5.366346  36.094330
1272     3367  3367_TANGIER-MEDITERRANEAN  -5.510966  35.886370
1883     5158                5158_vallvik  17.152920  61.207490
1921     2874            2874_GRENAA HAVN  10.928406  56.415765
2710     2311             2311_SUNDERLAND  -1.351393  54.919048
2762     2780              2780_GHAZAOUET  -1.865776  35.101685
2817     2760                2760_ALMERIA  -2.469166  36.825090
2818     3530                  3530_PLOCE  17.432616  43.041215
2819     4148   4148_PUERTO VIEJO DE AZUA -70.836886  18.346550


#### DISTANCE MEASURE

In [ ]:
print('Diameter:{}'.format(nx.diameter(sub_G_gibraltar)))

Diameter:13


## Malacca Blockage

In [ ]:
# route_blockages_mal = pd.read_csv((data_path + 'route_blockages_mal.csv'))  

In [ ]:
G_without_malacca = Build_Cut_Graph(df, route_blockages_mal)

Removing Edges:  41%|████      | 601/1476 [00:00<00:00, 6000.33it/s]

Before Pruning: 
 Nodes:  2912 Edges:  29838


Removing Edges: 100%|██████████| 1476/1476 [00:00<00:00, 5366.34it/s]


 After Pruning: 
 Nodes:  2912 Edges:  28599


In [ ]:
G_without_malacca.graph['name'] = 'Malacca Blocked'

In [ ]:
info_malacca = nx.info(G_without_malacca)
print(info_malacca)

Name: Malacca Blocked
Type: Graph
Number of nodes: 2912
Number of edges: 28599
Average degree:  19.6422


In [ ]:
df_nodes_malacca = pd.DataFrame(data = dict(G_without_malacca.degree()), index = range(1)).T.reset_index()
df_nodes_malacca.columns = ['nodes', 'k']
df_nodes_malacca = df_nodes_malacca.sort_values('k', ascending=False)
df_nodes_malacca[:10]

,nodes,k
11,1975,548
19,2005,469
450,2787,434
56,3128,388
433,2560,333
10,1973,308
50,2667,296
455,2866,277
399,2110,261
82,7007,261


#### SHORTEST PATHS

In [ ]:
#print(nx.dijkstra_path_length(G_without_malacca, 2998 ,2050, weight = 'distance'))
#print(nx.dijkstra_path(G_without_malacca, 1550 ,2050, weight = 'distance'))

In [ ]:
conn_groups_malacca = list(nx.connected_components(G_without_malacca))
len(conn_groups_malacca)

43

In [ ]:
subcomponent_malacca = max(conn_groups_malacca, key=len)
sub_G_malacca = nx.subgraph(G_without_malacca, subcomponent_malacca)
len(subcomponent_malacca)

2849

In [ ]:
avg_shortest_path_malacca = nx.average_shortest_path_length(sub_G_malacca, weight='distance')
avg_shortest_path_malacca

1770.9607027689449

#### CENTRALITY

In [ ]:
bw_cent_malacca = nx.betweenness_centrality(sub_G_malacca, weight = 'distance')
d_cent_malacca = nx.degree_centrality(sub_G_malacca)
cl_cent_malacca = nx.closeness_centrality(sub_G_malacca, distance = 'distance')
eig_cent_malacca = nx.eigenvector_centrality(sub_G_malacca, weight = 'distance')

In [ ]:
top_bw_cent_malacca = get_top_keys(bw_cent_malacca,10)
top_cl_cent_malacca = get_top_keys(cl_cent_malacca,10)
top_eig_cent_malacca = get_top_keys(eig_cent_malacca,10)
top_d_cent_malacca = get_top_keys(d_cent_malacca, 10)

In [ ]:
print('bw_cent')
print(pd.DataFrame(bw_cent_malacca.values()).describe())
print('\nd_cent')
print(pd.DataFrame(d_cent_malacca.values()).describe())
print('\ncl_cent')
print(pd.DataFrame(cl_cent_malacca.values()).describe())
print('\neig_cent')
print(pd.DataFrame(eig_cent_malacca.values()).describe())

bw_cent
                 0
count  2849.000000
mean      0.005254
std       0.027919
min       0.000000
25%       0.000000
50%       0.000012
75%       0.001399
max       0.659585

d_cent
                 0
count  2849.000000
mean      0.007039
std       0.012020
min       0.000351
25%       0.001404
50%       0.003160
75%       0.007725
max       0.192416

cl_cent
                 0
count  2849.000000
mean      0.000693
std       0.000248
min       0.000048
25%       0.000496
50%       0.000714
75%       0.000911
max       0.001056

eig_cent
                  0
count  2.849000e+03
mean   7.261457e-03
std    1.727357e-02
min    3.798432e-25
25%    2.815975e-05
50%    9.644707e-04
75%    5.965436e-03
max    1.743088e-01


In [ ]:
nodes_malacca = port_node[port_node['port_id'].isin(list(sub_G_malacca.nodes))]

for i, node in nodes_malacca.iterrows():
    sub_G_malacca.nodes[node['port_id']].update(node[1:].to_dict())
sub_G_malacca.nodes(data=True)

NodeDataView({26: {'port_name': '26_Victoria', 'x': -123.32715191091728, 'y': 48.402783083729446}, 1940: {'port_name': '1940_QINGDAO GANG', 'x': 120.22133924391858, 'y': 36.1114580135}, 537: {'port_name': '537_Victoria', 'x': -123.4493705745, 'y': 48.403568258}, 756: {'port_name': '756_Phatthaya Harbour', 'x': 100.77095414178388, 'y': 12.916740649000001}, 1550: {'port_name': '1550_Tangshan Gang', 'x': 119.06712856620985, 'y': 38.991341473999995}, 1812: {'port_name': '1812_Lianyungang Gang', 'x': 119.92488383459992, 'y': 34.8721305135}, 1934: {'port_name': '1934_Vancouver', 'x': -123.2597273115, 'y': 49.1586226685}, 1939: {'port_name': '1939_Dalian Gang', 'x': 121.73994207831859, 'y': 38.96239100955427}, 1945: {'port_name': '1945_TIANJIN XIN GANG', 'x': 118.03203396429149, 'y': 38.9173794765}, 1949: {'port_name': '1949_Port of Brisbane', 'x': 153.26802961287123, 'y': -27.0444596093148}, 1973: {'port_name': '1973_JOHOR', 'x': 104.08482446459018, 'y': 1.4279557760000001}, 1975: {'port_nam

In [ ]:
print('Top 10 places for betweenness centrality:')
for node_id in top_bw_cent_malacca:
    print(sub_G_malacca.nodes[node_id]['port_name'],'         ', bw_cent_malacca[node_id])

Top 10 places for betweenness centrality:
6868_nan           0.6595847491741751
7144_nan           0.5080059139721291
621_KUALA SUNGAI LINGGI PORT LIMIT           0.47185880662870044
1342_Gökçeada Harbour           0.3925783793703603
2824_mobile           0.3728784586969134
28291_nan           0.37160124199334604
23844_nan           0.3141925957937194
1975_TANJUNG PELEPAS           0.28288327354242393
842_Çanakkale Harbour           0.22512239376753768
2866_HANSWEERT           0.22271571099876472


In [ ]:
print('Top 10 places for degree centrality:')
for node_id in top_d_cent_malacca:
    print(sub_G_malacca.nodes[node_id]['port_name'],'         ', d_cent_malacca[node_id])

Top 10 places for degree centrality:
1975_TANJUNG PELEPAS           0.19241573033707865
2005_Shanghai Gang           0.16467696629213482
2787_EUROPA POINT           0.1523876404494382
3128_HONG KONG           0.13623595505617977
2560_EUROPOORT           0.11692415730337079
1973_JOHOR           0.10814606741573034
2667_fujairah           0.10393258426966293
2866_HANSWEERT           0.09726123595505617
7007_GWANGYANG HANG           0.09164325842696629
2110_Unknown           0.09164325842696629


In [ ]:
print('Top 10 places for closeness centrality:')
for node_id in top_cl_cent_malacca:
    print(sub_G_malacca.nodes[node_id]['port_name'],'         ', cl_cent_malacca[node_id])

Top 10 places for closeness centrality:
6868_nan           0.0010561473260856395
6869_nan           0.0010558900607090295
2075_PORT KLANG           0.0010553438758729259
1975_TANJUNG PELEPAS           0.0010552387123845364
7019_nan           0.0010551923302537123
6859_nan           0.001054955473509444
6856_nan           0.001054638016278369
6839_nan           0.001054239785034417
4043_DUMAI           0.0010541885478615347
25027_Xiazhimen Kou Outside Anchorage for Ore Vessel           0.0010537590043008093


In [ ]:
print('Top 10 places for eigenvector centrality:')
for node_id in top_eig_cent_malacca:
    print(sub_G_malacca.nodes[node_id]['port_name'],'         ', eig_cent_malacca[node_id])

Top 10 places for eigenvector centrality:
16816_Pacific Anchorage           0.1743087635161494
3269_BALBOA           0.17088759820341518
4734_PUERTO CRISTOBAL           0.15687115346406683
1934_Vancouver           0.15530828484558729
1975_TANJUNG PELEPAS           0.1516024139169457
7007_GWANGYANG HANG           0.150024544781525
2005_Shanghai Gang           0.14516675282581046
1973_JOHOR           0.1450360794192564
3099_RESERVE           0.14379418325908616
2171_Port of Newcastle           0.1341775015723863


#### CLUSTERING

In [ ]:
#avg of local clustering of a node, which is triangles that actually exist over all possible triangles in its neighborhood. 
avg_clust_malacca = nx.average_clustering(sub_G_malacca, weight='distance')
avg_clust_malacca

0.013513516889045061

In [ ]:
# number of pissible triangles
triadic_closure = nx.transitivity(sub_G_malacca)
print("Triadic closure:", triadic_closure)

Triadic closure: 0.22688486760775634


#### ISOLATES


In [ ]:
nx.number_of_isolates(G_without_malacca)

4

In [ ]:
isolated_malacca = nx.isolates(G_without_malacca)
print(port_node[port_node['port_id'].isin(list(isolated_malacca))])

      port_id                 port_name          x          y
2729     2599    2599_OGUENDJO TERMINAL   8.727052  -1.172190
2732     3549            3549_CAP LOPEZ   8.713756  -0.630555
2733     3603       3603_CEIBA TERMINAL   9.229674   1.408080
2743    50259  50259_Ash Shihr Terminal  49.524476  14.682465


#### DISTANCE MEASURE

In [ ]:
print('Diameter:{}'.format(nx.diameter(sub_G_gibraltar)))

Diameter:13


## Dover Blockage

In [ ]:
# route_blockages_dov = pd.read_csv((data_path + 'route_blockages_dov.csv'))  

In [ ]:
G_without_dover = Build_Cut_Graph(df, route_blockages_dov)

Removing Edges:  37%|███▋      | 607/1656 [00:00<00:00, 6065.94it/s]

Before Pruning: 
 Nodes:  2912 Edges:  29838


Removing Edges: 100%|██████████| 1656/1656 [00:00<00:00, 6228.23it/s]


 After Pruning: 
 Nodes:  2912 Edges:  28430


In [ ]:
G_without_dover.graph['name'] = 'Dover Blocked'

In [ ]:
info_dover = nx.info(G_without_dover)
print(info_dover)

Name: Dover Blocked
Type: Graph
Number of nodes: 2912
Number of edges: 28430
Average degree:  19.5261


In [ ]:
df_nodes_dover = pd.DataFrame(data = dict(G_without_dover.degree()), index = range(1)).T.reset_index()
df_nodes_dover.columns = ['nodes', 'k']
df_nodes_dover = df_nodes_dover.sort_values('k', ascending=False)
df_nodes_dover[:10]

,nodes,k
11,1975,875
19,2005,479
56,3128,400
450,2787,393
76,6868,363
10,1973,356
50,2667,340
82,7007,271
8,1945,260
77,6869,255


#### SHORTEST PATHS

In [ ]:
#print(nx.dijkstra_path_length(G_without_dover, 2998 ,2050, weight = 'distance'))
#print(nx.dijkstra_path(G_without_dover, 1550 ,2050, weight = 'distance'))

In [ ]:
conn_groups_dover = list(nx.connected_components(G_without_dover))
len(conn_groups_dover)

45

In [ ]:
subcomponent_dover = max(conn_groups_dover, key=len)
sub_G_dover = nx.subgraph(G_without_dover, subcomponent_dover)
len(subcomponent_dover)

2845

In [ ]:
avg_shortest_path_dover = nx.average_shortest_path_length(sub_G_dover, weight='distance')
avg_shortest_path_dover

1894.3198439596833

#### CENTRALITY

In [ ]:
bw_cent_dover = nx.betweenness_centrality(sub_G_dover, weight = 'distance')
d_cent_dover = nx.degree_centrality(sub_G_dover)
cl_cent_dover = nx.closeness_centrality(sub_G_dover, distance = 'distance')
eig_cent_dover = nx.eigenvector_centrality(sub_G_dover, weight = 'distance')

In [ ]:
top_bw_cent_dover = get_top_keys(bw_cent_dover,10)
top_cl_cent_dover = get_top_keys(cl_cent_dover,10)
top_eig_cent_dover = get_top_keys(eig_cent_dover,10)
top_d_cent_dover = get_top_keys(d_cent_dover, 10)

In [ ]:
print('bw_cent')
print(pd.DataFrame(bw_cent_dover.values()).describe())
print('\nd_cent')
print(pd.DataFrame(d_cent_dover.values()).describe())
print('\ncl_cent')
print(pd.DataFrame(cl_cent_dover.values()).describe())
print('\neig_cent')
print(pd.DataFrame(eig_cent_dover.values()).describe())

bw_cent
                 0
count  2845.000000
mean      0.004541
std       0.022880
min       0.000000
25%       0.000000
50%       0.000007
75%       0.001402
max       0.610419

d_cent
                 0
count  2845.000000
mean      0.007017
std       0.012778
min       0.000352
25%       0.001406
50%       0.003165
75%       0.007736
max       0.307665

cl_cent
                 0
count  2845.000000
mean      0.000651
std       0.000242
min       0.000046
25%       0.000435
50%       0.000669
75%       0.000879
max       0.000990

eig_cent
                  0
count  2.845000e+03
mean   6.978934e-03
std    1.740387e-02
min    6.703722e-26
25%    1.260897e-05
50%    7.334794e-04
75%    6.177564e-03
max    3.769275e-01


In [ ]:
nodes_dover = port_node[port_node['port_id'].isin(list(sub_G_dover.nodes))]

for i, node in nodes_dover.iterrows():
    sub_G_dover.nodes[node['port_id']].update(node[1:].to_dict())
sub_G_dover.nodes(data=True)

NodeDataView({26: {'port_name': '26_Victoria', 'x': -123.32715191091728, 'y': 48.402783083729446}, 1940: {'port_name': '1940_QINGDAO GANG', 'x': 120.22133924391858, 'y': 36.1114580135}, 537: {'port_name': '537_Victoria', 'x': -123.4493705745, 'y': 48.403568258}, 756: {'port_name': '756_Phatthaya Harbour', 'x': 100.77095414178388, 'y': 12.916740649000001}, 1550: {'port_name': '1550_Tangshan Gang', 'x': 119.06712856620985, 'y': 38.991341473999995}, 1812: {'port_name': '1812_Lianyungang Gang', 'x': 119.92488383459992, 'y': 34.8721305135}, 1934: {'port_name': '1934_Vancouver', 'x': -123.2597273115, 'y': 49.1586226685}, 1939: {'port_name': '1939_Dalian Gang', 'x': 121.73994207831859, 'y': 38.96239100955427}, 1945: {'port_name': '1945_TIANJIN XIN GANG', 'x': 118.03203396429149, 'y': 38.9173794765}, 1949: {'port_name': '1949_Port of Brisbane', 'x': 153.26802961287123, 'y': -27.0444596093148}, 1973: {'port_name': '1973_JOHOR', 'x': 104.08482446459018, 'y': 1.4279557760000001}, 1975: {'port_nam

In [ ]:
print('Top 10 places for betweenness centrality:')
for node_id in top_bw_cent_dover:
    print(sub_G_dover.nodes[node_id]['port_name'],'         ', bw_cent_dover[node_id])

Top 10 places for betweenness centrality:
6869_nan           0.6104185125654691
6868_nan           0.465662200890187
1594_Tekirdag Port           0.25021779750694206
2052_DURBAN           0.23613331136806517
2111_Richards Bay           0.232817990544051
1592_Izmir Liman           0.2320776521700844
3026_NEMRUT LIMANI BAY           0.22706892790197555
7026_nan           0.22505000314142912
7144_nan           0.20877319524897187
621_KUALA SUNGAI LINGGI PORT LIMIT           0.18026113933450186


In [ ]:
print('Top 10 places for degree centrality:')
for node_id in top_d_cent_dover:
    print(sub_G_dover.nodes[node_id]['port_name'],'         ', d_cent_dover[node_id])

Top 10 places for degree centrality:
1975_TANJUNG PELEPAS           0.30766526019690577
2005_Shanghai Gang           0.16842475386779185
3128_HONG KONG           0.14064697609001406
2787_EUROPA POINT           0.13818565400843882
6868_nan           0.12763713080168776
1973_JOHOR           0.12517580872011252
2667_fujairah           0.11954992967651196
7007_GWANGYANG HANG           0.09528832630098454
1945_TIANJIN XIN GANG           0.09142053445850915
6869_nan           0.08966244725738397


In [ ]:
print('Top 10 places for closeness centrality:')
for node_id in top_cl_cent_dover:
    print(sub_G_dover.nodes[node_id]['port_name'],'         ', cl_cent_dover[node_id])

Top 10 places for closeness centrality:
6869_nan           0.0009897099984491659
6868_nan           0.00098964486455289
2052_DURBAN           0.0009893309940270183
2075_PORT KLANG           0.0009892301196238044
7019_nan           0.0009890961033111564
3419_AL JUBAYL           0.0009890191663702683
6859_nan           0.0009889750122871884
7100_nan           0.0009889218721307715
6820_No 1           0.0009888243408608078
1975_TANJUNG PELEPAS           0.0009887320405276975


In [ ]:
print('Top 10 places for eigenvector centrality:')
for node_id in top_eig_cent_dover:
    print(sub_G_dover.nodes[node_id]['port_name'],'         ', eig_cent_dover[node_id])

Top 10 places for eigenvector centrality:
1975_TANJUNG PELEPAS           0.37692746883578865
6868_nan           0.23075206979246
6869_nan           0.1869922805498308
3099_RESERVE           0.16811341865009494
2667_fujairah           0.16103979555058576
2787_EUROPA POINT           0.1379321970486556
28317_No.3           0.13323587256679045
2110_Unknown           0.12554174870697038
1973_JOHOR           0.12345445897379821
7144_nan           0.12174300811089421


#### CLUSTERING

In [ ]:
#avg of local clustering of a node, which is triangles that actually exist over all possible triangles in its neighborhood. 
avg_clust_dover = nx.average_clustering(sub_G_dover, weight='distance')
avg_clust_dover

0.014613701527924777

In [ ]:
# number of pissible triangles
triadic_closure = nx.transitivity(sub_G_dover)
print("Triadic closure:", triadic_closure)

Triadic closure: 0.20921738413185023


#### ISOLATES


In [ ]:
nx.number_of_isolates(G_without_dover)

6

In [ ]:
isolated_dover = nx.isolates(G_without_dover)
print(port_node[port_node['port_id'].isin(list(isolated_dover))])

      port_id         port_name          x          y
1710    12991         12991_nan   1.734488  51.007500
1883     5158      5158_vallvik  17.152920  61.207490
1921     2874  2874_GRENAA HAVN  10.928406  56.415765
2710     2311   2311_SUNDERLAND  -1.351393  54.919048
2723     2650  2650_fredrikstad  10.961776  59.188265
2772     5179        5179_Sauda   6.361665  59.645325


#### DISTANCE MEASURE

In [ ]:
print('Diameter:{}'.format(nx.diameter(sub_G_dover)))

Diameter:12


## Panama Blockage

In [ ]:
# route_blockages_pan = pd.read_csv((data_path + 'route_blockages_pan.csv'))  

In [41]:
G_without_panama = Build_Cut_Graph(df, route_blockages_pan)

Removing Edges: 100%|██████████| 972/972 [00:00<00:00, 6043.12it/s]

Before Pruning: 
 Nodes:  2912 Edges:  29838

 After Pruning: 
 Nodes:  2912 Edges:  29129


In [ ]:
G_without_panama.graph['name'] = 'Panama Blocked'

In [ ]:
info_panama = nx.info(G_without_panama)
print(info_panama)

Name: Panama Blocked
Type: Graph
Number of nodes: 2912
Number of edges: 29129
Average degree:  20.0062


In [ ]:
df_nodes_panama = pd.DataFrame(data = dict(G_without_panama.degree()), index = range(1)).T.reset_index()
df_nodes_panama.columns = ['nodes', 'k']
df_nodes_panama = df_nodes_panama.sort_values('k', ascending=False)
df_nodes_panama[:10]

,nodes,k
11,1975,878
19,2005,476
450,2787,434
56,3128,397
76,6868,371
10,1973,356
50,2667,341
433,2560,335
455,2866,276
82,7007,268


#### SHORTEST PATHS

In [ ]:
#print(nx.dijkstra_path_length(G_without_panama, 2998 ,2050, weight = 'distance'))
#print(nx.dijkstra_path(G_without_panama, 1550 ,2050, weight = 'distance'))

In [42]:
conn_groups_panama = list(nx.connected_components(G_without_panama))
len(conn_groups_panama)

45

In [43]:
subcomponent_panama = max(conn_groups_panama, key=len)
sub_G_panama = nx.subgraph(G_without_panama, subcomponent_panama)
len(subcomponent_panama)

2846

In [ ]:
avg_shortest_path_panama = nx.average_shortest_path_length(sub_G_panama, weight='distance')
avg_shortest_path_panama

1789.7361133570014

#### CENTRALITY

In [44]:
bw_cent_panama = nx.betweenness_centrality(sub_G_panama, weight = 'distance')
d_cent_panama = nx.degree_centrality(sub_G_panama)
cl_cent_panama = nx.closeness_centrality(sub_G_panama, distance = 'distance')
eig_cent_panama = nx.eigenvector_centrality(sub_G_panama, weight = 'distance')

In [45]:
top_bw_cent_panama = get_top_keys(bw_cent_panama,10)
top_cl_cent_panama = get_top_keys(cl_cent_panama,10)
top_eig_cent_panama = get_top_keys(eig_cent_panama,10)
top_d_cent_panama = get_top_keys(d_cent_panama, 10)

In [46]:
print('bw_cent')
print(pd.DataFrame(bw_cent_panama.values()).describe())
print('\nd_cent')
print(pd.DataFrame(d_cent_panama.values()).describe())
print('\ncl_cent')
print(pd.DataFrame(cl_cent_panama.values()).describe())
print('\neig_cent')
print(pd.DataFrame(eig_cent_panama.values()).describe())

bw_cent
                 0
count  2846.000000
mean      0.004738
std       0.024545
min       0.000000
25%       0.000000
50%       0.000008
75%       0.001397
max       0.606382

d_cent
                 0
count  2846.000000
mean      0.007185
std       0.013249
min       0.000351
25%       0.001406
50%       0.003163
75%       0.007733
max       0.308612

cl_cent
                 0
count  2846.000000
mean      0.000697
std       0.000257
min       0.000046
25%       0.000478
50%       0.000729
75%       0.000932
max       0.001052

eig_cent
                  0
count  2.846000e+03
mean   6.910763e-03
std    1.742752e-02
min    6.252206e-26
25%    2.321198e-05
50%    8.848055e-04
75%    6.210875e-03
max    3.711127e-01


In [47]:
nodes_panama = port_node[port_node['port_id'].isin(list(sub_G_panama.nodes))]

for i, node in nodes_panama.iterrows():
    sub_G_panama.nodes[node['port_id']].update(node[1:].to_dict())
sub_G_panama.nodes(data=True)

NodeDataView({26: {'port_name': '26_Victoria', 'x': -123.32715191091728, 'y': 48.402783083729446}, 1940: {'port_name': '1940_QINGDAO GANG', 'x': 120.22133924391858, 'y': 36.1114580135}, 537: {'port_name': '537_Victoria', 'x': -123.4493705745, 'y': 48.403568258}, 756: {'port_name': '756_Phatthaya Harbour', 'x': 100.77095414178388, 'y': 12.916740649000001}, 1550: {'port_name': '1550_Tangshan Gang', 'x': 119.06712856620985, 'y': 38.991341473999995}, 1812: {'port_name': '1812_Lianyungang Gang', 'x': 119.92488383459992, 'y': 34.8721305135}, 1934: {'port_name': '1934_Vancouver', 'x': -123.2597273115, 'y': 49.1586226685}, 1939: {'port_name': '1939_Dalian Gang', 'x': 121.73994207831859, 'y': 38.96239100955427}, 1945: {'port_name': '1945_TIANJIN XIN GANG', 'x': 118.03203396429149, 'y': 38.9173794765}, 1949: {'port_name': '1949_Port of Brisbane', 'x': 153.26802961287123, 'y': -27.0444596093148}, 1973: {'port_name': '1973_JOHOR', 'x': 104.08482446459018, 'y': 1.4279557760000001}, 1975: {'port_nam

In [48]:
print('Top 10 places for betweenness centrality:')
for node_id in top_bw_cent_panama:
    print(sub_G_panama.nodes[node_id]['port_name'],'         ', bw_cent_panama[node_id])

Top 10 places for betweenness centrality:
6869_nan           0.6063817638465588
6868_nan           0.45970871986533485
1594_Tekirdag Port           0.3052059155772088
2052_DURBAN           0.28483163148020435
2111_Richards Bay           0.2826057509535074
1592_Izmir Liman           0.28202907363326485
3026_NEMRUT LIMANI BAY           0.2793881732948717
1342_Gökçeada Harbour           0.25011901848679674
842_Çanakkale Harbour           0.22473557626946872
2866_HANSWEERT           0.2223215896816039


In [49]:
print('Top 10 places for degree centrality:')
for node_id in top_d_cent_panama:
    print(sub_G_panama.nodes[node_id]['port_name'],'         ', d_cent_panama[node_id])

Top 10 places for degree centrality:
1975_TANJUNG PELEPAS           0.3086115992970123
2005_Shanghai Gang           0.16731107205623902
2787_EUROPA POINT           0.15254833040421795
3128_HONG KONG           0.13954305799648506
6868_nan           0.1304042179261863
1973_JOHOR           0.12513181019332162
2667_fujairah           0.11985940246045695
2560_EUROPOORT           0.11775043936731108
2866_HANSWEERT           0.09701230228471003
7007_GWANGYANG HANG           0.09420035149384887


In [50]:
print('Top 10 places for closeness centrality:')
for node_id in top_cl_cent_panama:
    print(sub_G_panama.nodes[node_id]['port_name'],'         ', cl_cent_panama[node_id])

Top 10 places for closeness centrality:
6869_nan           0.0010517716595274792
6868_nan           0.0010516961746138633
2052_DURBAN           0.0010514064650990747
2075_PORT KLANG           0.001051229726577575
7019_nan           0.0010510783614592492
3419_AL JUBAYL           0.0010509895808764537
6859_nan           0.001050941528272981
7100_nan           0.0010508816168786211
6820_No 1           0.0010507695771927374
1975_TANJUNG PELEPAS           0.001050666786084156


In [51]:
print('Top 10 places for eigenvector centrality:')
for node_id in top_eig_cent_panama:
    print(sub_G_panama.nodes[node_id]['port_name'],'         ', eig_cent_panama[node_id])

Top 10 places for eigenvector centrality:
1975_TANJUNG PELEPAS           0.3711126605484885
6868_nan           0.2452696132416226
6869_nan           0.1995802162434132
3099_RESERVE           0.18618187825882987
2667_fujairah           0.15395909814751363
2787_EUROPA POINT           0.14447887343434856
28317_No.3           0.14239995804254385
2560_EUROPOORT           0.14030406882171145
7144_nan           0.13183389978476903
2110_Unknown           0.131705683762251


#### CLUSTERING

In [52]:
#avg of local clustering of a node, which is triangles that actually exist over all possible triangles in its neighborhood. 
avg_clust_panama = nx.average_clustering(sub_G_panama, weight='distance')
avg_clust_panama

0.0148346181796939

In [53]:
# number of pissible triangles
triadic_closure = nx.transitivity(sub_G_panama)
print("Triadic closure:", triadic_closure)

Triadic closure: 0.2046622369700747


#### ISOLATES


In [54]:
nx.number_of_isolates(G_without_panama)

7

In [55]:
isolated_panama = nx.isolates(G_without_panama)
print(port_node[port_node['port_id'].isin(list(isolated_panama))])

      port_id                                port_name          x          y
57       3269                              3269_BALBOA -79.575915   8.951900
89      16816                  16816_Pacific Anchorage -79.485323   8.873569
1279     3729                        3729_PUERTO COLON -79.884956   9.370420
1334     3000                  3000_BAHIA DE LAS MINAS -79.821016   9.392585
1779    15192          15192_Hazardous Cargo Anchorage -79.933478   9.376085
2593    27498                                27498_nan -79.823565   9.429898
2888    24779  24779_Zona de Fondeo Guanta Pertigalete -64.578105  10.266503


#### DISTANCE MEASURE

In [56]:
print('Diameter:{}'.format(nx.diameter(sub_G_panama)))

Diameter:12


## Total Blockage

In [57]:
# route_blockages_total = pd.read_csv((data_path + 'route_blockages_total.csv'))  

In [58]:
G_blocked = Build_Cut_Graph(df, route_blockages_total)

Removing Edges:   0%|          | 0/7988 [00:00<?, ?it/s]

Before Pruning: 
 Nodes:  2912 Edges:  29838


Removing Edges: 100%|██████████| 7988/7988 [00:01<00:00, 6061.15it/s]


 After Pruning: 
 Nodes:  2912 Edges:  24306


In [59]:
G_blocked.graph['name'] = 'Total Blocked'

In [60]:
info_blocked = nx.info(G_blocked)
print(info_blocked)

Name: Total Blocked
Type: Graph
Number of nodes: 2912
Number of edges: 24306
Average degree:  16.6937


In [61]:
df_nodes_blocked = pd.DataFrame(data = dict(G_blocked.degree()), index = range(1)).T.reset_index()
df_nodes_blocked.columns = ['nodes', 'k']
df_nodes_blocked = df_nodes_blocked.sort_values('k', ascending=False)
df_nodes_blocked[:10]

,nodes,k
11,1975,544
19,2005,466
56,3128,385
10,1973,307
82,7007,258
8,1945,253
80,6973,230
165,2171,200
220,3099,198
50,2667,193


#### SHORTEST PATHS

In [62]:
#print(nx.dijkstra_path_length(G_blocked, 2998 ,2050, weight = 'distance'))
#print(nx.dijkstra_path(G_blocked, 1550 ,2050, weight = 'distance'))

In [63]:
conn_groups_blocked = list(nx.connected_components(G_blocked))
len(conn_groups_blocked)

78

In [64]:
subcomponent_blocked = max(conn_groups_blocked, key=len)
sub_G_blocked = nx.subgraph(G_blocked, subcomponent_blocked)
len(subcomponent_blocked)


2809

In [65]:
avg_shortest_path_blocked = nx.average_shortest_path_length(sub_G_blocked, weight='distance')
avg_shortest_path_blocked

2129.7558073155715

#### CENTRALITY

In [66]:
bw_cent_blocked = nx.betweenness_centrality(sub_G_blocked, weight = 'distance')
d_cent_blocked = nx.degree_centrality(sub_G_blocked)
cl_cent_blocked = nx.closeness_centrality(sub_G_blocked, distance = 'distance')
eig_cent_blocked = nx.eigenvector_centrality(sub_G_blocked, weight = 'distance')

In [67]:
top_bw_cent_blocked = get_top_keys(bw_cent_blocked,10)
top_cl_cent_blocked = get_top_keys(cl_cent_blocked,10)
top_eig_cent_blocked = get_top_keys(eig_cent_blocked,10)
top_d_cent_blocked = get_top_keys(d_cent_blocked, 10)

In [79]:
print('bw_cent')
print(pd.DataFrame(bw_cent_blocked.values()).describe())
print('\nd_cent')
print(pd.DataFrame(d_cent_blocked.values()).describe())
print('\ncl_cent')
print(pd.DataFrame(cl_cent_blocked.values()).describe())
print('\neig_cent')
print(pd.DataFrame(eig_cent_blocked.values()).describe())

bw_cent
                 0
count  2809.000000
mean      0.005462
std       0.028488
min       0.000000
25%       0.000000
50%       0.000015
75%       0.001424
max       0.647775

d_cent
                 0
count  2809.000000
mean      0.006150
std       0.010215
min       0.000356
25%       0.001425
50%       0.002849
75%       0.007123
max       0.193732

cl_cent
                 0
count  2809.000000
mean      0.000558
std       0.000200
min       0.000047
25%       0.000389
50%       0.000554
75%       0.000748
max       0.000859

eig_cent
                  0
count  2.809000e+03
mean   6.427731e-03
std    1.774246e-02
min    4.954244e-25
25%    7.172547e-07
50%    1.560417e-04
75%    4.117898e-03
max    1.973083e-01


In [69]:
nodes_blocked = port_node[port_node['port_id'].isin(list(sub_G_blocked.nodes))]

for i, node in nodes_blocked.iterrows():
    sub_G_blocked.nodes[node['port_id']].update(node[1:].to_dict())
sub_G_blocked.nodes(data=True)

NodeDataView({26: {'port_name': '26_Victoria', 'x': -123.32715191091728, 'y': 48.402783083729446}, 1940: {'port_name': '1940_QINGDAO GANG', 'x': 120.22133924391858, 'y': 36.1114580135}, 537: {'port_name': '537_Victoria', 'x': -123.4493705745, 'y': 48.403568258}, 756: {'port_name': '756_Phatthaya Harbour', 'x': 100.77095414178388, 'y': 12.916740649000001}, 1550: {'port_name': '1550_Tangshan Gang', 'x': 119.06712856620985, 'y': 38.991341473999995}, 1812: {'port_name': '1812_Lianyungang Gang', 'x': 119.92488383459992, 'y': 34.8721305135}, 1934: {'port_name': '1934_Vancouver', 'x': -123.2597273115, 'y': 49.1586226685}, 1939: {'port_name': '1939_Dalian Gang', 'x': 121.73994207831859, 'y': 38.96239100955427}, 1945: {'port_name': '1945_TIANJIN XIN GANG', 'x': 118.03203396429149, 'y': 38.9173794765}, 1949: {'port_name': '1949_Port of Brisbane', 'x': 153.26802961287123, 'y': -27.0444596093148}, 1973: {'port_name': '1973_JOHOR', 'x': 104.08482446459018, 'y': 1.4279557760000001}, 1975: {'port_nam

In [70]:
print('Top 10 places for betweenness centrality:')
for node_id in top_bw_cent_blocked:
    print(sub_G_blocked.nodes[node_id]['port_name'],'         ', bw_cent_blocked[node_id])

Top 10 places for betweenness centrality:
6868_nan           0.6477746415402277
7144_nan           0.5185827149667549
621_KUALA SUNGAI LINGGI PORT LIMIT           0.46496650112610216
2824_mobile           0.3801850684643702
28291_nan           0.37726552564457805
1342_Gökçeada Harbour           0.3766007244810238
23844_nan           0.35081506652578975
7026_nan           0.2996662292173514
1975_TANJUNG PELEPAS           0.2502578007565539
2458_KISARAZU KO           0.17802639311367494


In [71]:
print('Top 10 places for degree centrality:')
for node_id in top_d_cent_blocked:
    print(sub_G_blocked.nodes[node_id]['port_name'],'         ', d_cent_blocked[node_id])

Top 10 places for degree centrality:
1975_TANJUNG PELEPAS           0.19373219373219375
2005_Shanghai Gang           0.16595441595441596
3128_HONG KONG           0.1371082621082621
1973_JOHOR           0.10933048433048434
7007_GWANGYANG HANG           0.09188034188034189
1945_TIANJIN XIN GANG           0.09009971509971511
6973_nan           0.08190883190883191
2171_Port of Newcastle           0.07122507122507123
3099_RESERVE           0.07051282051282051
2667_fujairah           0.06873219373219373


In [72]:
print('Top 10 places for closeness centrality:')
for node_id in top_cl_cent_blocked:
    print(sub_G_blocked.nodes[node_id]['port_name'],'         ', cl_cent_blocked[node_id])

Top 10 places for closeness centrality:
6868_nan           0.0008594869217524222
6869_nan           0.0008593187713554609
2075_PORT KLANG           0.0008589569900380666
7019_nan           0.0008588572764715459
1975_TANJUNG PELEPAS           0.0008588194377109108
6859_nan           0.0008586113721021691
6856_nan           0.0008584215169339451
621_KUALA SUNGAI LINGGI PORT LIMIT           0.0008583122918649487
4043_DUMAI           0.0008581892825769308
6839_nan           0.0008581707090035441


In [73]:
print('Top 10 places for eigenvector centrality:')
for node_id in top_eig_cent_blocked:
    print(sub_G_blocked.nodes[node_id]['port_name'],'         ', eig_cent_blocked[node_id])

Top 10 places for eigenvector centrality:
1975_TANJUNG PELEPAS           0.19730826848953445
1934_Vancouver           0.18727698751129493
1973_JOHOR           0.1865245497709324
2171_Port of Newcastle           0.18540521901590645
2005_Shanghai Gang           0.18172633751733666
7007_GWANGYANG HANG           0.17236679135421526
3128_HONG KONG           0.16243176051251337
3648_PORT HEDLAND           0.15642443096206765
3042_HAY POINT           0.14540753294550482
1945_TIANJIN XIN GANG           0.13385645032365812


#### CLUSTERING

In [74]:
#avg of local clustering of a node, which is triangles that actually exist over all possible triangles in its neighborhood. 
avg_clust_blocked = nx.average_clustering(sub_G_blocked, weight='distance')
avg_clust_blocked

0.011169805321356937

In [75]:
# number of pissible triangles
triadic_closure = nx.transitivity(sub_G_blocked)
print("Triadic closure:", triadic_closure)

Triadic closure: 0.2385398877775067


#### ISOLATES


In [76]:
nx.number_of_isolates(G_blocked)

33

In [77]:
isolated_blocked = nx.isolates(G_blocked)
print(port_node[port_node['port_id'].isin(list(isolated_blocked))])

      port_id                                port_name          x          y
57       3269                              3269_BALBOA -79.575915   8.951900
89      16816                  16816_Pacific Anchorage -79.485323   8.873569
450      2787                        2787_EUROPA POINT  -5.366346  36.094330
575      6946                                 6946_nan  56.354580  26.955025
1163     2425                    2425_Ajman Port Limit  55.447174  25.433304
1196     4031                        4031_RAS SHUKHIER  33.332824  28.094150
1272     3367               3367_TANGIER-MEDITERRANEAN  -5.510966  35.886370
1279     3729                        3729_PUERTO COLON -79.884956   9.370420
1334     3000                  3000_BAHIA DE LAS MINAS -79.821016   9.392585
1710    12991                                12991_nan   1.734488  51.007500
1779    15192          15192_Hazardous Cargo Anchorage -79.933478   9.376085
1883     5158                             5158_vallvik  17.152920  61.207490

#### DISTANCE MEASURE

In [78]:
print('Diameter:{}'.format(nx.diameter(sub_G_blocked)))

Diameter:15
